In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as anim
import random
import math

In [2]:
class Vertex:
    def __init__(self,label,x,y,blocked):
        self.xy=(x,y)
        self.label = self.xy
        self.color = 'white'
        self.dist = math.inf
        self.parent=None
        self.left=None
        self.right=None
        self.up=None
        self.down=None
        self.blocked=blocked #blocked contains the string characters of the grid

In [3]:
def find_vertex(x,y, graph):
    vt = Vertex('X',-1,-1,'X') #dummy vertex

    for v in graph:
        if graph[v].label==(x,y):
            if graph[v].blocked!='X':
                return graph[v]
            else:
                return vt

In [4]:
#helper function to populate graph with right, left,...
def build_graph(graph, rlen, clen):   

    for v in graph: #in graph
        # if y < (clen -1) assign right
        if graph[v].label[1]<(clen-1):
            if graph[v].blocked!='X':  # only going to find vertex if isn't blocked
                vt= find_vertex(graph[v].label[0], graph[v].label[1]+1, graph) #incrementing y
                if vt.label!=(-1,-1):
                    graph[v].right= vt #incrementing y
                    
        # if y > 0 assign left
        if graph[v].label[1]>0:
            if graph[v].blocked!='X': # only going to find vertex if isn't blocked
                vt= find_vertex(graph[v].label[0], graph[v].label[1]-1, graph) #decreasing y
                if vt.label!=(-1,-1):
                    graph[v].left= vt #decreasing y
                    
        #checking in the up for the entire grid, if x>0 assign up  
        if graph[v].label[0]>0:
            if graph[v].blocked!='X':  # only going to find vertex if isn't blocked
                vt= find_vertex(graph[v].label[0]-1, graph[v].label[1], graph) #decreasing x
                if vt.label!=(-1,-1):
                    graph[v].up= vt #decreasing x
                    
    #checking for down for entire grid, assign down except for last row, x<(rlen-1)
        if graph[v].label[0]<(rlen-1):
            if graph[v].blocked!='X':  # only going to find vertex if isn't blocked
                vt=graph[v].down= find_vertex(graph[v].label[0]+1, graph[v].label[1], graph) #increasing x 
                if vt.label!=(-1,-1):
                    graph[v].down=vt #increasing x
                    

In [5]:
def dfs_visit(lst, graph, p, s, d, time, dfound):  
    time = time + 1          #// white vertex u has just been discovered
    graph[s].dist = time
    graph[s].color = 'gray'
    if graph[d].color == 'green':
        return 
   
    adjlst=[] 
    if graph[s].up!=None:
        adjlst.append(graph[s].up)
        
    if graph[s].down!=None:
        adjlst.append(graph[s].down)
        
    if graph[s].left!=None:
        adjlst.append(graph[s].left)
        
    if graph[s].right!=None:
        adjlst.append(graph[s].right)
    
    for v in adjlst:  #// explore edge (u, v) # lst has adjacent points/node
        
        if ((v.color == 'white') and (v.label == d)):
            dfound = True
            graph[d].color = 'green'
            lst.append(((s[0],s[1]), (d[0],d[1]), 'Forward'))
            #print(lst)
            return dfound
                                
        if not dfound:
            if ((v.color == 'white') and (v.blocked != 'X')): # also check for blocked
                #here plot line between points s, v (potential path) 
                lst.append(((s[0],s[1]), (v.label[0],v.label[1]), 'Forward'))
                dfound = dfs_visit(lst, graph, s, v.label, d, time, dfound)
    
    if not dfound:
        graph[s].color = 'black'         # // blacken u; it is finished
        
        xpt = [s[0], p[0]] #graph[s].label # pt1 = node, pt2 = parent
        ypt = [s[1], p[1]]
        #here plot line in different color to backtrack between points s and its parent
        lst.append(((s[0],s[1]), (p[0],p[1]), 'Back'))

    
    return dfound

In [6]:
def Do_dfs_noplot(graph, lst):    
    #running DFS
    time = 0
    spoint=(-9,-9)
    dpoint=(-9,-9)
    dfound=False
    
    for u in graph:
        if graph[u].blocked == 'S':
            spoint=graph[u].label
        if graph[u].blocked=='D':
            dpoint=graph[u].label

    while (graph[dpoint].color != 'green'):
        dfs_visit(lst, graph, spoint, spoint, dpoint, time, dfound)  

In [7]:
def anim_dfs(grid):
    global lst
    graph = {}
    for xpt,row in enumerate(grid):
        for ypt,col in enumerate(row):
            graph[xpt,ypt] = Vertex(grid[xpt][ypt], xpt, ypt, grid[xpt][ypt])

    # populate graph (right, left, up, down...)
    build_graph(graph, len(grid), (len(grid[0])))    
    
    lst=[]
    Do_dfs_noplot(graph, lst)

    plt.rcParams["animation.html"] = "jshtml"
    plt.rcParams['figure.dpi'] = 150  
    plt.ioff()
    
    # Setting limits for x and y axis

    fig, ax = plt.subplots() 

    ax.set_xlim(-1,(len(grid[0])-1)+1, 1) 
    ax.set_ylim((len(grid)-1)+1,-1, 1)
    
    ax.xaxis.set_ticks(np.arange(-1, (len(grid[0])-1)+1, 1))
    ax.yaxis.set_ticks(np.arange(-1,(len(grid)-1)+1, 1))
    
    plt.grid()
    
    # lets plot the grid points
    for xpt,row in enumerate(grid):
        for ypt,col in enumerate(row):
            x = xpt
            y = ypt
           
            if grid[x][y]=='X':
                plt.plot(y, x, marker="o", markersize=10,color='black')  
            elif grid[x][y]=='S':
                plt.plot(y, x, marker="o", markersize=10,color='green')  
            elif grid[x][y]=='D':
                plt.plot(y, x, marker="o", markersize=10,color='red') 
    line = None
    len_list = len(lst)
    def animate_grid(t):
        pt1 = lst[t][0]
        pt2 = lst[t][1]
        c = lst[t][2]
        
        if c == 'Forward':
            line, = ax.plot([pt1[1],pt2[1]],[pt1[0],pt2[0]], color='red',lw=5) 
            
        else: 
            line, = ax.plot([pt1[1],pt2[1]],[pt1[0],pt2[0]], color='blue',lw=5) 
        return 
    
   
    return anim.FuncAnimation(fig, animate_grid, frames = np.arange(0,len_list,1), interval=500)

In [8]:
 grid = [
     '--------------------',
     '-XX-X-XXX-----X-----',
     '-XX---S-------X-----',
     '-XXXXXXXX-----XXXX--',
     '----XXXXX-----------',
     'XXX-XXXXX-----X-----',
     'X---XXXXXXXXXXXXXX-X',
     'X-XXX---------DX----',
     'X-----X----X-XX-----'
     ]

anim_dfs(grid)